In [1]:
from lxml import html
import urllib
from urllib import request
from bs4 import BeautifulSoup
import re
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import requests
import lxml.html
from requests import get
import pandas as pd
import numpy as np
import time
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / size)     
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

## Slobodna Evropa 

Based on the data structure we need to load more content on the page for a maximum of three times -> extract all links and remove duplicates.

In [62]:
months_2018 = ['11','12']
months_2019 = ['1','2']
days_1 = ['1','2','3','4','5','6','7','8','9']
days_2 = range(10,32)
days_2 = ["{:02d}".format(x) for x in days_2]
days = days_1 + days_2
categories = ['500', '504']

In [ ]:
driver = webdriver.Chrome()
se_urls_2018 = []
for a in categories:
    for i in months_2018:
        for y in log_progress(days):
            try: 
                url = 'https://www.slobodnaevropa.org/z/' + a + '/2018/' + i + '/' + y
                driver.get(url)
                python_button = driver.find_element_by_css_selector('.btn__text')
                count = 3
                while count > 1:
                    python_button.click()
                    count -= 1
                soup = BeautifulSoup(driver.page_source, 'lxml')
                for link in soup.findAll('a', attrs={'href': re.compile("^/a/")}):
                    se_urls_2018.append(link.get('href'))
            except:
                pass

In [ ]:
se_urls = se_urls_2018 + se_urls_2019
clean_links = ['https://www.slobodnaevropa.org' + x for x in se_urls]
clean_links = list(set(clean_links))
len(clean_links)
np.save("clean_links_se.npy", clean_links)
clean_links = np.load("clean_links_se.npy").tolist()

In [ ]:
driver = webdriver.Chrome()
se_urls_2019 = []
for a in categories:
    for i in months_2019:
        for y in log_progress(days):
            try: 
                url = 'https://www.slobodnaevropa.org/z/' + a + '/2019/' + i + '/' + y
                driver.get(url)
                python_button = driver.find_element_by_css_selector('.btn__text')
                count = 3
                while count > 1:
                    python_button.click()
                    count -= 1
                soup = BeautifulSoup(driver.page_source, 'lxml')
                for link in soup.findAll('a', attrs={'href': re.compile("^/a/")}):
                    se_urls_2019.append(link.get('href'))
            except:
                pass

In [439]:
pages = []
for page in log_progress(clean_links): 
    new_pages = requests.get(page)
    pages.append(new_pages)
trees = [page.content for page in pages]
soups = [BeautifulSoup(tree, 'lxml') for tree in trees]
titles = []
for soup in log_progress(soups):
    try:
        title = soup.find("div", {"class" : "col-title col-xs-12 col-md-10 pull-right"}).findAll('h1')
        titles.append(title)
    except AttributeError:
        titles.append('problem')
texts = []
for soup in log_progress(soups):
    try:
        text = soup.find("div", {"class" : "col-xs-12 col-sm-12 col-md-8 col-lg-8 pull-left bottom-offset content-offset"}).findAll('p')
        texts.append(text)
    except AttributeError:
        texts.append('problem')
dates = []
for soup in log_progress(soups):
    try:
        date = soup.find("div", {"class" : "published"}).findAll('time')
        dates.append(date)
    except AttributeError:
        dates.append('problem')

In [441]:
all_data.isna().sum()

title    0
date     0
text     0
dtype: int64

In [442]:
len(all_data)

1794

In [ ]:
all_data = pd.DataFrame(
    {'title': pd.Series(titles),
     'date' : pd.Series(dates),
     'text': pd.Series(texts),
    })

In [452]:
all_data.to_csv('se.csv', sep='\t', encoding='utf-8')

## VOA

In [3]:
months_2018 = ['11','12']
months_2019 = ['1','2']
days_1 = ['1','2','3','4','5','6','7','8','9']
days_2 = range(10,32)
days_2 = ["{:02d}".format(x) for x in days_2]
days = days_1 + days_2
categories = ['2086', '2079', '2087', '2080', '2078', '2084']

In [ ]:
driver = webdriver.Chrome()
voa_urls_2018 = []
for a in categories:
    for i in months_2018:
        for y in log_progress(days):
            try: 
                url = 'https://www.glasamerike.net/z/' + a + '/2018/' + i + '/' + y
                driver.get(url)
                python_button = driver.find_element_by_css_selector('.btn__text')
                count = 3
                while count > 1:
                    python_button.click()
                    count -= 1
                soup = BeautifulSoup(driver.page_source, 'lxml')
                for link in soup.findAll('a', attrs={'href': re.compile("^/a/")}):
                    voa_urls_2018.append(link.get('href'))
            except:
                pass

In [ ]:
driver = webdriver.Chrome()
voa_urls_2019 = []
for a in categories:
    for i in months_2019:
        for y in log_progress(days):
            try: 
                url = 'https://www.glasamerike.net/z/' + a + '/2019/' + i + '/' + y
                driver.get(url)
                python_button = driver.find_element_by_css_selector('.btn__text')
                count = 3
                while count > 1:
                    python_button.click()
                    count -= 1
                soup = BeautifulSoup(driver.page_source, 'lxml')
                for link in soup.findAll('a', attrs={'href': re.compile("^/a/")}):
                    voa_urls_2019.append(link.get('href'))
            except:
                pass

In [133]:
np.save("voa_2018links.npy", voa_urls_2018)

In [ ]:
voa_urls = voa_urls_2018 + voa_urls_2019
clean_links = ['https://www.glasamerike.net' + x for x in voa_urls]
clean_links = list(set(clean_links))

In [25]:
np.save("voa_clean_links.npy", clean_links)

In [ ]:
pages = []
for page in log_progress(clean_links): 
    new_pages = requests.get(page)
    pages.append(new_pages)
trees = [page.content for page in pages]
soups = [BeautifulSoup(tree, 'lxml') for tree in trees]

titles = []
for soup in log_progress(soups):
    try:
        title = soup.find("div", {"class" : "col-title col-xs-12 col-md-10 pull-right"}).findAll('h1')
        titles.append(title)
    except AttributeError:
        titles.append("problem")
texts = []
for soup in log_progress(soups):
    try:
        text = soup.find("div", {"class" : "col-xs-12 col-sm-12 col-md-8 col-lg-8 pull-left bottom-offset content-offset"}).findAll('p')
        texts.append(text)
    except AttributeError:
        texts.append("problem")
dates = []
for soup in log_progress(soups):
    try:
        date = soup.find("div", {"class" : "published"}).findAll('time')
        dates.append(date)
    except AttributeError:
        dates.append("problem")
categories = []
for soup in log_progress(soups):
    try:
        category = soup.find("div", {"class" : "category"})
        categories.append(category)
    except AttributeError:
        categories.append("problem")

In [363]:
all_data = pd.DataFrame(
    {'title': pd.Series(titles),
     'date' : pd.Series(dates),
     'text': pd.Series(texts),
     'category' : pd.Series(categories),
    })

In [369]:
all_data.isna().sum()

title       0
date        0
text        0
category    7
dtype: int64

Since Category does not throw an Attribute Error it returns seven missing values where all others return "problem". We just drop those rows

In [371]:
all_data = all_data[pd.notnull(all_data['category'])]

In [388]:
all_data.to_csv('voa.csv', sep='\t', encoding='utf-8')

## N1

In [ ]:
n1_urls = []
url = 'http://rs.n1info.com/Svet'
driver.get(url)
try: 
    i = 0 
    while i < 80:
        i += 1
        print(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        python_button = driver.find_element_by_css_selector('.js-btn-load-more')
        python_button.click()
        time.sleep(3)
        if i == 80:
            break 
except Exception as exc:
    print(exc)
            
soup = BeautifulSoup(driver.page_source, 'lxml')
for link in soup.findAll('a', attrs={'href': re.compile("/Svet/")}):
    n1_urls.append(link.get('href'))
    
url = 'http://rs.n1info.com/Region'
driver.get(url)
try: 
    i = 0 
    while i < 30:
        i += 1
        print(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        python_button = driver.find_element_by_css_selector('.js-btn-load-more')
        python_button.click()
        time.sleep(3)
        if i == 30:
            break 
except Exception as exc:
    print(exc)
            
soup = BeautifulSoup(driver.page_source, 'lxml')
for link in soup.findAll('a', attrs={'href': re.compile("/Region/")}):
    n1_urls.append(link.get('href'))

url = 'http://rs.n1info.com/Arhiva-Vesti'
driver.get(url)
try: 
    i = 0 
    while i < 100:
        i += 1
        print(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        python_button = driver.find_element_by_css_selector('.js-btn-load-more')
        python_button.click()
        time.sleep(3)
        if i == 100:
            break 
except Exception as exc:
    print(exc)
            
soup = BeautifulSoup(driver.page_source, 'lxml')
for link in soup.findAll('a', attrs={'href': re.compile("/Vesti/")}):
    n1_urls.append(link.get('href'))
    
url = 'http://rs.n1info.com/Kultura'
driver.get(url)
try: 
    i = 0 
    while i < 25:
        i += 1
        print(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        python_button = driver.find_element_by_css_selector('.js-btn-load-more')
        python_button.click()
        time.sleep(3)
        if i == 25:
            break 
except Exception as exc:
    print(exc)
            
soup = BeautifulSoup(driver.page_source, 'lxml')
for link in soup.findAll('a', attrs={'href': re.compile("/Kultura/")}):
    n1_urls.append(link.get('href'))

In [158]:
len(n1_urls)
np.save('n1_urls.npy', n1_urls)
np.save('copy_n1_urls.npy', copy)

In [73]:
svet = [s for s in n1_urls if "/Svet/" in s]

In [83]:
region = [s for s in n1_urls if "/Region/" in s]

In [110]:
kultura = [s for s in n1_urls if "/Kultura/" in s]
len(kultura)

531

In [123]:
vesti = [s for s in n1_urls if "/Vesti/" in s]
len(vesti)

5790

In [136]:
vesti = [s for s in n1_urls if "/Vesti/" in s]
len(vesti)

6280

In [141]:
len(list(set(vesti)))

3058

In [ ]:
pages = []
for page in log_progress(clean_links): 
    new_pages = requests.get(page)
    pages.append(new_pages)
trees = [page.content for page in pages]

In [7]:
soups = []
for tree in log_progress(trees):
    try: 
        soup = BeautifulSoup(tree, 'lxml')
        soups.append(soup)
    except:
        pass

In [ ]:
titles = []
for soup in log_progress(soups):
    try:
        title = soup.find("div", {"class" : "single-article-header"}).findAll('h1')
        titles.append(title)
    except AttributeError:
        titles.append("problem")
        
texts = []
for soup in log_progress(soups):
    try:
        text = soup.find("div", {"class" : "single-article-content js_resizeFont"}).findAll('p')
        texts.append(text)
    except AttributeError:
        texts.append("problem")

categories = []
for soup in log_progress(soups):
    try:
        category = soup.find("div", {"class" : "article-info-cnt"}).findAll('p')
        categories.append(category)
    except AttributeError:
        categories.append("problem")
        
dates = []
for soup in log_progress(soups):
    try:
        date = soup.find("span", {"class" : "date"})
        dates.append(date)
    except AttributeError:
        categories.append("problem")

In [67]:
all_data = pd.DataFrame(
    {'title': titles,
     'date' : dates,
     'text': texts,
     'category' : categories
    })

In [78]:
all_data.to_csv('n1.csv', sep=',', encoding='utf-8')